# “网络数据采集” 课程

# 第4章 Web内容解析



## 案例名称

爬取新浪财经网http://finance.sina.com.cn/stock/，各股票公司每日公告（爬取股票分析所需语料）

## 案例内容

本案例用于爬取新浪财经股票公司公告。

基本流程如下：

1. 输入开始日期、结束日期，作为查询条件，之后程序后计算之间有多少天，之后生成每一天的日期，以每天的日期作为查询条件提交到 http://vip.stock.finance.sina.com.cn/corp/view/vCB_BulletinGather.php?gg_date=&ftype=0 进行查询。

2. 程序采用了多线程技术加速爬取过程。每个线程都将首先执行spiderOneGroupDays；

3. 调用 spiderOneDay

4. 调用 spiderOnePage

5. 调用 spiderOnePiece

In [ ]:
## coding:utf-8
# 爬取新浪财经网股票公司每日公告
# 提供日期即可  eg: 2017-02-21
import os
import math
import time
import datetime
import requests
import threading
from lxml import etree


# 爬取一条公告并保存
def spiderOnePiece(iurl,headers,datetime,base_dir,filename):
    # 去除文件名中的非法字符
    invaild=['*','\\','/',':','\"','<','>','|','?']
    for c in invaild:
        if c in filename:
            filename=filename.replace(c,'')
    print("    公告链接为：",iurl)
    try:
        response=requests.get(iurl,headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        
        page=etree.HTML(response.text)
        content=page.xpath('//*[@id="content"]/pre')
        if len(content)==0:
            return
        content=content[0].text
        print(content[:20])
        filename = os.path.join(base_dir,datetime,filename)
        with open(filename,'w+') as f:
            f.write(content)
    except Exception as e:
        print("spiderOnePiece error:",e)

# 爬取一页
def spiderOnePage(url,headers,datetime,base_dir):
    website='http://vip.stock.finance.sina.com.cn'
    try:
        response=requests.get(url,headers=headers)
        response.encoding = response.apparent_encoding
        response.raise_for_status()
    
        page=etree.HTML(response.text)
        trList=page.xpath(r'//*[@id="wrap"]/div[@class="Container"]/table/tbody/tr')

        print("当前页面共有公告{}条".format(len(trList) ))
        if len(trList)==1:  # 爬取结束  该行（对不起没有相关记录）
            return 0

        #按日期建立文件目录
        a_dirByDate = os.path.join(base_dir,datetime)
        if not os.path.exists(a_dirByDate):  # 创建日期文件夹
            os.mkdir(a_dirByDate)

        for item in trList:
            aUrl=item.xpath('th/a[1]')
            title=aUrl[0].text    # 公告标题
            href=aUrl[0].attrib['href']   # 公告uri
            href=website+href    # 公告url

            atype=item.xpath('td[1]')[0].text # 公告类型
            print("准备爬取公告{}".format(title))
            a_filename = title+atype+'.txt'
            print("    存储位置：",a_filename)
            spiderOnePiece(href,headers,datetime,base_dir,a_filename)
        return 1
    except Exception as e:
        print(e)

# 爬取一天
def spiderOneDay(url,headers,datetime,base_dir='company_annoucement'):
    url=url.replace('#datetime#',datetime)  # 填充日期
    flag=1   # 爬取成功标志
    index=1  # 起始页
    
    log_dir = os.path.join(base_dir,'log')
    print(log_dir)
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    log_file = os.path.join(log_dir,datetime+'.log')    
    
    with open(log_file,'w') as f:
        while flag:
            t_url=url+str(index)
            try:
                flag=spiderOnePage(t_url,headers,datetime,base_dir)
            except Exception as e:
                print('spiderOneDay err:',e)
                flag=0
            finally:
                if flag:
                    print('%s page_%d load success,continue.' %(datetime,index))
                    f.write('%s_page_%d load success.\n' %(datetime,index))
                    f.flush()
                else:
                    print('%s page_%d load fail,end.' %(datetime,index))
                    f.write('%s_page_%d load failed.\n' %(datetime,index))
                    f.flush()
                index+=1
    

# 爬取一组天股票公司的数据
def spiderOneGroupDays(url,headers,date_group,base_dir):
    for idate in date_group:
        try:

            spiderOneDay(url,headers,idate,base_dir)
            print('%s has load success.over.' %idate)
        except Exception as e:
            print('spiderOneGroupDays err:',e)
            continue


# 获取指定起始日期[包含]--结束日期[包含]之间的日期  
def getBetweenDay(begin_date,end_date):
    date_list=[]
    begin_date=datetime.datetime.strptime(begin_date,'%Y-%m-%d')
    # 现在的日期
    now_date=datetime.datetime.strptime(time.strftime('%Y-%m-%d',time.localtime(time.time())),'%Y-%m-%d')
    end_date=datetime.datetime.strptime(end_date,'%Y-%m-%d')
    # 如果给出的结束日期大于现在的日期  则将今天的日期作为结束日期
    if end_date > now_date:
        end_date = now_date
    while begin_date <= end_date:
        date_str=begin_date.strftime('%Y-%m-%d')
        date_list.append(date_str)
        begin_date+=datetime.timedelta(days=1)
    return date_list

# 将date_list 平均分成threadNum组  最后一组可能较少
def split_date_list(date_list,threadNum):
    # length=(len(date_list)/threadNum if len(date_list)%threadNum==0 else len(date_list)/threadNum+1)
    length=int(math.ceil(len(date_list)*1.0/threadNum))
    return [date_list[m:m+length] for m in range(0,len(date_list),length)]

def main():
    headers = {
        "Accept-Language": "zh-CN,zh;q=0.8", 
        "Accept-Encoding": "gzip, deflate, sdch", 
        "Host": "vip.stock.finance.sina.com.cn", 
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8", 
        "Upgrade-Insecure-Requests": "1", 
        "Connection": "keep-alive", 
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36"
    }
    
    url='http://vip.stock.finance.sina.com.cn/corp/view/vCB_BulletinGather.php?gg_date=#datetime#&page='
    
    # 创建数据与日志的保存文件夹
    base_dir = "company_announcements"
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    
    # datetime='2017-02-19'
    # spiderOneDay(url,headers,datetime,log_path)
    #symbol = input("请输入A股股票的代码/名称/拼音")
    begin_date = input("请输入需查询的开始日期（例如：2017-01-01）：")
    end_date = input("请输入需查询的结束日期（例如：2017-01-31）：")
    # begin_date[包含]-->end_date[包含] 之间的所有date
    date_list = getBetweenDay(begin_date,end_date)
    print('%s 到 %s，共 %d 天。' % (begin_date,end_date,len(date_list)))
    
    # begin_date[包含]-->end_date[包含] 之间的所有date
    date_list=getBetweenDay(begin_date,end_date)
    print('%s-%s:%d days.' %(begin_date,end_date,len(date_list)))

    cut_date_list=split_date_list(date_list,4)
    print(cut_date_list)

    threads=[]
    for dgroup in cut_date_list:
        t=threading.Thread(target=spiderOneGroupDays,args=(url,headers,dgroup,base_dir))
        threads.append(t)

    # 开始线程
    for t in threads:
        t.start()

    # 等待所有线程结束  阻塞主线程
    for t in threads:
        t.join()
    print('all load success...')
    
if __name__ == '__main__':
    main()

## 案例结论

这个案例较为综合的演示了：

- 使用requests库爬取sina股票网站的公司公告

- 使用lxml的xpath解析文档内容，提取公告文本

- 使用多线程技术，加速爬取。

